# Trabalho 1 de Segurança em Computação - Tarefa 1
Semestre: 2025/2

Alune(s): Lu Mendonça Emery Cade (2017100210)



## Especificações



### Objetivo Geral
Este trabalho consolida alguns conceitos estudados na disciplina Segurança em Computação, por meio da implementação de um serviço de certificação digital privado. Na construção deste serviço você deverá criar e configurar um ambiente Docker + Nginx seguro, com suporte a HTTPS, por meio de duas abordagens em tarefas distintas:

### Tarefa 1
Implementação de um serviço de emissão de certificado digital por uma CA Privada (PKI Própria). O serviço deverá estar associado à operação de uma CA Raiz e uma CA Intermediária, responsáveis por emitir e validar certificados. Você também deverá instalar a CA raiz no seu navegador como autoridade confiável, permitindo que a conexão HTTPS seja reconhecida como segura. Nesta tarefa, a implementação deverá ser feita usando a linguagem Python.




## Implementação

In [ ]:
import os

# Criar a pasta 'arquivos' se ela não existir
os.makedirs('arquivos', exist_ok=True)
print("[Pasta 'arquivos/' criada]")

*   Criação da CA Raiz (autoassinada): criar chaves públicas/privadas RSA 4096 bits, criar certificado da CA Raiz e assinar com a própria chave privada (autoassinatura).
    *   Exportar .key e .pem da CA Raiz



In [1]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
import datetime

In [2]:
# Gerar a chave privada
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
)

# Obter a chave pública
public_key = private_key.public_key()

# Salvar a chave privada em arquivos/root.key
with open("arquivos/root.key", "wb") as root_file:
    root_file.write(private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.BestAvailableEncryption(b'this-is-a-password-and-it-passwords-things'),
    ))
print("[Chave raiz gerada e criada]")

In [3]:
# Definir o subject (que também vai ser o issuer)
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"BR"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"ES"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"UFES"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"Root-CA"),
])

# Construir o certificado
cert_builder = x509.CertificateBuilder().subject_name(
    subject
).issuer_name(
    issuer
).public_key(
    public_key
).serial_number(
    x509.random_serial_number()
).not_valid_before(
    datetime.datetime.now(datetime.UTC)
).not_valid_after(
    # 10 anos de validade pareceu um número razoável
    datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=3650)
)

In [4]:
# Adicionar extensões recomendadas
cert_builder = cert_builder.add_extension(
    x509.BasicConstraints(ca=True, path_length=1), critical=True,
)
cert_builder = cert_builder.add_extension(
    x509.KeyUsage(key_cert_sign=True, crl_sign=True, digital_signature=False,
                  content_commitment=False, key_encipherment=False,
                  data_encipherment=False, key_agreement=False,
                  encipher_only=False, decipher_only=False), critical=True,
)
cert_builder = cert_builder.add_extension(
    x509.SubjectKeyIdentifier.from_public_key(public_key), critical=False,
)
print("[Certificado raiz feito]")

In [5]:
# Assinar o certificado
certificate = cert_builder.sign(private_key, hashes.SHA256())

# Salvar o certificado em arquivos/root.pem
with open("arquivos/root.pem", "wb") as root_file:
    root_file.write(certificate.public_bytes(serialization.Encoding.PEM))
print("[Certificado raiz assinado e criado]")



*   Criação da CA Intermediária (assinada pela Raiz): criar chaves pública/privada RSA 4096 bits, criar certificado da CA Intermediária e assinar assinar o certificado gerado com a CA Raiz;
    *   Exportar .key e .pem da CA Intermediária



In [6]:
# 1. Gerar a nova chave privada
intermediate_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
)

# 2. Obter a chave pública correspondente
intermediate_public_key = intermediate_private_key.public_key()

# 3. Salvar a chave privada em arquivos/intermediate.key
with open("arquivos/intermediate.key", "wb") as intermediate_file:
    intermediate_file.write(intermediate_private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.BestAvailableEncryption(b'this-is-a-second-password-it-also-passwords-things'),
    ))
print("[Chave intermediária gerada e criada]")

In [7]:
# Carregar a chave privada da Root (para assinar)
with open("arquivos/root.key", "rb") as root_file:
    root_private_key = serialization.load_pem_private_key(
        root_file.read(),
        password=b'this-is-a-password-and-it-passwords-things',
    )

# Carregar o certificado da Raiz (para pegar o issuer name)
with open("arquivos/root.pem", "rb") as root_file:
    root_cert = x509.load_pem_x509_certificate(root_file.read())
print("[Chave raiz carregada]")

In [8]:
# Definir o subject da Intermediária
intermediate_subject = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"BR"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"ES"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"UFES"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"Intermediate-CA"),
])

# O issuer é o subject da CA Raiz
intermediate_issuer = root_cert.subject

cert_builder = x509.CertificateBuilder().subject_name(
    intermediate_subject
).issuer_name(
    intermediate_issuer  # assinado pela Root
).public_key(
    intermediate_public_key # chave pública da Intermediária
).serial_number(
    x509.random_serial_number()
).not_valid_before(
    datetime.datetime.now(datetime.UTC)
).not_valid_after(
    # 3 anos parece razoável pra intermediária
    datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=1095)
)

# Extensões da CA Intermediária
cert_builder = cert_builder.add_extension(
    # path_length=0 significa que
    #   - pode assinar certificados finais (servidores),
    #   - não pode criar outras CAs.
    x509.BasicConstraints(ca=True, path_length=0), critical=True,
)
cert_builder = cert_builder.add_extension(
    x509.KeyUsage(key_cert_sign=True, crl_sign=True, digital_signature=False,
                  content_commitment=False, key_encipherment=False,
                  data_encipherment=False, key_agreement=False,
                  encipher_only=False, decipher_only=False), critical=True,
)
cert_builder = cert_builder.add_extension(
    x509.SubjectKeyIdentifier.from_public_key(intermediate_public_key), critical=False,
)
cert_builder = cert_builder.add_extension(
    # O AuthorityKeyIdentifier aponta para o SubjectKeyIdentifier da Root
    x509.AuthorityKeyIdentifier.from_issuer_public_key(root_cert.public_key()), critical=False,
)
print("[Certificado intermediário feito]")

In [ ]:
# Assinar com a chave privada da Root
intermediate_certificate = cert_builder.sign(root_private_key, hashes.SHA256())

# Salvar o certificado em arquivos/intermediate.pem
with open("arquivos/intermediate.pem", "wb") as intermediate_file:
    intermediate_file.write(intermediate_certificate.public_bytes(serialization.Encoding.PEM))
print("[Certificado intermediário assinado e criado]")

In [9]:
# Carrega a CA intermediária
def load_intermediate_ca():
    print("[Carregando CA Intermediária]")
    with open("arquivos/intermediate.key", "rb") as intermediate_file:
        inter_key = serialization.load_pem_private_key(
            intermediate_file.read(),
            password=b'this-is-a-second-password-it-also-passwords-things',
        )
    with open("arquivos/intermediate.pem", "rb") as intermediate_file:
        inter_cert = x509.load_pem_x509_certificate(intermediate_file.read())
    return inter_key, inter_cert

In [10]:
# ------- ~ use sua criatividade (tm)~ -------
SERVER_NAME = "localhost"
ORGANIZATION = "UFES"
UNIT = "LabSeg II"
COUNTRY = "BR"
STATE = "ES"
# --------------------------------------------

In [11]:
def create_server_certificate():
    # 1. Carregar a CA
    ca_key, ca_cert = load_intermediate_ca()

    # 2. Gera a própria chave
    print(f"[Gerando chave privada para {SERVER_NAME}]")
    server_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=4096,
    )

    # 3. Cra o CSR
    print("[Gerando CSR]")
    csr_subject = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, COUNTRY),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, STATE),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, ORGANIZATION),
        x509.NameAttribute(NameOID.ORGANIZATIONAL_UNIT_NAME, UNIT),
        x509.NameAttribute(NameOID.COMMON_NAME, SERVER_NAME),
    ])

    csr = x509.CertificateSigningRequestBuilder().subject_name(
        csr_subject
    ).add_extension(
        x509.SubjectAlternativeName([x509.DNSName(SERVER_NAME)]),
        critical=False,
    ).sign(server_key, hashes.SHA256())

    # 4. Geração do CRT assinando com a CA
    print("[Assinando o certificado do servidor]")

    # Construindo o certificado final
    cert_builder = x509.CertificateBuilder().subject_name(
        csr.subject # Mantém o nome do request
    ).issuer_name(
        ca_cert.subject # issuer é a CA Intermediária
    ).public_key(
        csr.public_key() # A chave pública vem do request do servidor
    ).serial_number(
        x509.random_serial_number()
    ).not_valid_before(
        datetime.datetime.now(datetime.UTC)
    ).not_valid_after(
        # 1 ano parece ok
        datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=365)
    )

    # Adicionando extensões recomendadas
    cert_builder = cert_builder.add_extension(
        x509.BasicConstraints(ca=False, path_length=None), critical=True,
    )
    cert_builder = cert_builder.add_extension(
        x509.SubjectKeyIdentifier.from_public_key(csr.public_key()), critical=False,
    )
    cert_builder = cert_builder.add_extension(
        x509.AuthorityKeyIdentifier.from_issuer_public_key(ca_key.public_key()), critical=False,
    )
    # Repetindo o SAN no final
    cert_builder = cert_builder.add_extension(
        x509.SubjectAlternativeName([x509.DNSName(SERVER_NAME)]), critical=False,
    )

    # Assinar com a Intermediária
    server_cert = cert_builder.sign(ca_key, hashes.SHA256())

    # 5. Gerando os arquivos
    print("[Salvando server.key e server.crt]")

    # Salvar a chave do servidor (sem senha para o Nginx não reclamar)
    with open("arquivos/server.key", "wb") as f:
        f.write(server_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption(),
        ))

    # Salvar o CRT
    with open("arquivos/server.crt", "wb") as f:
        f.write(server_cert.public_bytes(serialization.Encoding.PEM)) # certificado do servidor
        f.write(ca_cert.public_bytes(serialization.Encoding.PEM)) # certificado da intermediária

In [12]:
create_server_certificate()

[Carregando CA Intermediária]
[Gerando chave privada para localhost]
[Gerando CSR]
[Assinando o certificado do servidor]
[Salvando server.key e server.crt]


*   Validar a cadeia de certificação no navegador e no terminal usando Python.

está no arquivo validate.py